In [ ]:
!unzip ssbd-release.zip -d testing-ssbd

In [2]:
#parse through xml files and download youtube links
#!unzip ssbd-release.zip -d testing-ssbd
!mkdir headbanging
!mkdir spinning
!mkdir armflapping
!pip install pytube
!pip install scikit-video
from pytube import YouTube
import os
import lxml.etree
from bs4 import BeautifulSoup
import skvideo.io
directory = 'testing-ssbd/Annotations'
times={}
badvideos=["5BVFjqo0FUY",
           "xeKKMkVgNPU",
           "ehlLfMossUY",
           "TH5mlAhdw00",
           "5WTHMIJ_61I",
           "vtVnrV_jog0",
           "zuoD4tEtYyk",
           "Pqd9Vu-juPI",
           "-rC-ab0nzxY",
           "w96PxUcoo58",
           "T9rbit_oiJA",
           "Pa-pdBF4FFA",
           "wu5fgT-LkHs",
           "cxqsde-6V-c",
           "WcsZ9eMQ5Zs"]
def isgood(url):
  score=1
  for i in badvideos:
    if i in url:
      score=0
  return score

def split(time):
  a=int((len(time)-1)/2)
  return int(time[0:a])

def main(xmlfile):
    # use the parse() function to load and parse an XML file
    soup = BeautifulSoup(open(xmlfile), "xml")
    url=soup.find('url').string
    if (isgood(url)==1):
      yt = YouTube(url)
      file=yt.streams.filter(progressive = True, file_extension = "mp4").first().download(soup.find('category').string)
      times[(file[9:len(file)])]=soup.find('time').string
      print(times[(file[9:len(file)])])
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        main(f)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 56 kB 3.2 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.3 MB 7.5 MB/s 
0002:0047
0003-0020
0025:0028
0010:0019
0016:0030
0002:0015
0004:0008
0016:0022
0003:0007
0001:0015
0100:0107
0004:0030
0001:0110
0100:0134
0002-0150
0001:0047
0001:0235
0004:0009
18:24
0108:0123
11:16
0002:0014
0032:0046
0003:0010
0055:0104
0008:0015
0010:0017
0018:0021
0001:0007
0006:0012
0010:0020
0018:0027
0001:0010
0019:0022
0002:0006
0105:0125
0056:0103
0051-0058
0001:0004
0002:0058
0001:0023
0003:0030
0005:0015
0003:0017
0003:0007
0006:0014
0006-0018
0004-0012
0007:0016
0030:0040
0002:0007
0002-0059
0008:0018
0017:0023
0009:0026
22:27
0134:0138
0003-0028
0002:0020


In [3]:
from absl import logging
import tensorflow as tf
import tensorflow_hub as hub
!pip install -q git+https://github.com/tensorflow/docs 
from tensorflow_docs.vis import embed
logging.set_verbosity(logging.ERROR)
import random
import re
import os
import tempfile
import ssl
import cv2
import glob
import numpy as np
import pickle
from keras.utils import np_utils
import skvideo.io
import imageio
from IPython import display
_VIDEO_LIST = None
from urllib import request  # requires python3
os.environ["CUDA_VISIBLE_DEVICES"]="1"
T = 16 # depth of frames
def split(time):
  a=int((len(time)-1)/2)
  return int(time[0:a]),int(time[a+1:2*a+1])
def crop_center_square(frame):
    y, x = frame.shape[0:2]
    min_dim = min(y, x)
    start_x = (x // 2) - (min_dim // 2)
    start_y = (y // 2) - (min_dim // 2)
    return frame[start_y:start_y+min_dim,start_x:start_x+min_dim]
def load_video(path, resize=(224, 224)):
  cap = cv2.VideoCapture(path)
  frames = []
  fps = cap.get(cv2.CAP_PROP_FPS)
  a=int(fps*split(times[path])[0])
  b=min(int(cap.get(cv2.CAP_PROP_FRAME_COUNT)),int(fps*split(times[path])[1]))
  cap.set(cv2.CAP_PROP_POS_FRAMES, a)
  try:
    while True:
      ret, frame = cap.read()
      if not ret:
        break
      frame = crop_center_square(frame)
      frame = cv2.resize(frame, resize)
      frame = frame[:, :, [2, 1, 0]]
      frames.append(frame)
      if len(frames) == min(100,int(round(b-a))):
        break
  finally:
    cap.release()
  return np.array(frames) / 255.0
label_map=['armflapping','headbanging','spinning']
def loaddata():
  labels=[]
  data=[] 
  for filename in os.listdir('armflapping'):
    f = os.path.join('armflapping', filename)
    if os.path.isfile(f) and f.endswith(".mp4"):
      labels.append(0)
      data.append(load_video(f))
  for filename in os.listdir('headbanging'):
    f = os.path.join('headbanging', filename)
    if os.path.isfile(f) and f.endswith(".mp4"):
      labels.append(1)
      data.append(load_video(f))
  for filename in os.listdir('spinning'):
    f = os.path.join('spinning', filename)
    if os.path.isfile(f) and f.endswith(".mp4"):
      labels.append(2)
      data.append(load_video(f))
  return np.array(data),np.array(labels)      
def trim1(video):
    start = np.random.randint(0, video.shape[0] - (T+1)) #changed shape[1] to shape[0]
    end = start + T
    return np.array(video[start:end, :, :])
def trim(video,i):
    try:
        start = np.random.randint(0, video.shape[0] - (T+1)) #changed shape[1] to shape[0]
        end = start + T
        return np.array(video[start:end, :, :, :])
    except:
        print("video.shape[0] ",video.shape[0],i)
        return np.random.rand(16,224,224,3)
T = 16
def train_array(X_train):
    X = []
    i = 0
    for ele in X_train:
        i+=1
        if ele is not None:
            ele = trim(ele,i)
        else:
            ele =  np.random.rand(16,224,224,3)
        X.append(ele)
    print("from train array",len(X))
    return np.array(X)
import matplotlib.pyplot as plt
from keras.optimizers import SGD # optimizer
from keras.regularizers import l2
from keras.layers import Dropout
from keras.layers import Conv3D
from keras.layers import Conv3DTranspose
def train_i3d(X_train,Y_train,X_test,Y_test):
    i3d = "https://tfhub.dev/deepmind/i3d-kinetics-400/1"
    hub_layer = hub.KerasLayer(i3d,input_shape=[None,224,224,3],trainable=False)
    model = tf.keras.Sequential()
    print("Here ------> ",type(model))
    model.add(Conv3D(8,kernel_size=(2,2,16),strides=(1, 1, 1),input_shape=(None,224,224,3),padding="same"))
    model.add(Conv3DTranspose(8,kernel_size=(2,2,16),strides=(1, 1, 1),input_shape=(None,224,224,3),padding="same"))
    model.add(tf.keras.layers.Dense(32,activation='relu'))
    model.add(Conv3D(16,kernel_size=(2,2,16),strides=(1, 1, 1),input_shape=(None,224,224,3),padding="same"))
    model.add(tf.keras.layers.Dense(3,activation='sigmoid'))
    model.add(hub_layer)
    model.add(tf.keras.layers.Dense(3, activation='softmax')) # change as per no of classes 20-5-21
    opt = SGD(learning_rate=(0.002)) # 2*10^-3
    model.compile(loss='categorical_crossentropy',optimizer=opt, metrics=['accuracy']) #20-5-21
    [print(i.shape, i.dtype) for i in model.inputs]
    [print(o.shape, o.dtype) for o in model.outputs]
    [print(l.name, l.input_shape, l.dtype) for l in model.layers]
    return model   

In [ ]:
from sklearn.model_selection import train_test_split
if __name__=="__main__":
    directory = "/tf/i3d_finetuning/multi_action_dataset/train"
    n_classes = 3
    Xin, Yin = loaddata()
    #print(Xin.shape,Yin.shape)
    Yin = np_utils.to_categorical(Yin,n_classes)
    #print(Xin[2].shape,Yin[1].shape)
    vid = Xin[2]
    #print(vid.shape)
    #print(Xin.shape,Yin.shape)
    X_train, X_test, Y_train, Y_test = train_test_split(Xin, Yin, test_size=0.25, random_state=4)
    #print("coming back to main Split done size is:{0} and {1} ".format(X_train.shape,Y_train.shape))
    model = train_i3d(X_train,Y_train,X_test,Y_test)
    #print("From Main.py",type(model))
    model.summary()
    tf.keras.backend.set_floatx('float16')
    #print(X_train[0].shape)
    #print()
    X_train = train_array(Xin)
    #print()
    #print("again in main before split",X_train.shape)
    X_train, X_test, Y_train, Y_test = train_test_split(X_train, Yin, test_size=0.25, random_state=4)
    #print("X_train shape",X_train.shape)
    #print("Y_train shape",Y_train.shape)
    #print("X_test shape",X_test.shape)
    #print("Y_test shape",Y_test.shape)
    #print(type(X_test[0][0][0][0][0]))
    history = model.fit(X_train, Y_train, validation_data=(X_test, Y_test), batch_size=8, epochs=50, verbose=1,shuffle=True)
    model.save_weights("/tf/i3d_finetuning/models/multiclass6_STA_npdi_i3d_lr_0.002_50_epochs_batch32.h5")
    loss, acc = model.evaluate(X_test, Y_test, verbose=0)
    print('Test loss:', loss)
    print('Test accuracy:', acc)
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()
    plt.savefig("/tf/i3d_finetuning/models/multiclass6_STA_accuracy_npdi_50.png")
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper right')
    plt.show()
    plt.savefig("/tf/i3d_finetuning/models/multiclass6_STA_loss_npdi_50.png")   

<ipython-input-3-c8ab30db7a0e>:73: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(data),np.array(labels)


Here ------>  <class 'keras.engine.sequential.Sequential'>
(None, None, 224, 224, 3) <dtype: 'float32'>
(None, 3) <dtype: 'float32'>
conv3d (None, None, 224, 224, 3) float32
conv3d_transpose (None, None, 224, 224, 8) float32
dense (None, None, 224, 224, 8) float32
conv3d_1 (None, None, 224, 224, 32) float32
dense_1 (None, None, 224, 224, 16) float32
keras_layer (None, None, 224, 224, 3) float32
dense_2 (None, 400) float32
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d (Conv3D)             (None, None, 224, 224, 8  1544      
                             )                                   
                                                                 
 conv3d_transpose (Conv3DTra  (None, None, 224, 224, 8  4104     
 nspose)                     )                                   
                                                                 
 dense (Dense)            

In [ ]:
print(Yin)

[[1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]]
